# Taller de Machine Learning con R
## Bloque 6: Comparación de modelos

Hacemos el preprocesamiento de la base de datos

In [1]:
library('titanic')
library('plyr')
library('caret')
library('e1071')
library('arm')

Loading required package: lattice
Loading required package: ggplot2
Loading required package: MASS
Loading required package: Matrix
Loading required package: lme4

arm (Version 1.10-1, built: 2018-4-12)

Working directory is C:/Users/luisa/OneDrive/Cursos/R/Machine Learning con R 1



In [2]:
database<-titanic_train #Llamar base
database<-database[sample(1:nrow(database)),2:12] #Aleatorizar
database<-na.omit(database) #Quitar los renglones con NAs
database$Sex<-revalue(factor(database$Sex),
            c('male'=0,'female'=1)) #Cambiamos a numérico
database$Embarked<-revalue(factor(database$Embarked),
            c('S'=0,'C'=1,'Q'=2)) #Cambiamos a numérico

Generamos ahora la base de validación

In [3]:
variables<-c('Fare','Survived','Age','Sex')
database$Survived<-as.factor(database$Survived)#Para no tener 
                                                #problemas con caret
train_perc<-0.8
val_perc<-0.2
train_size<-dim(database)[1]*train_perc #tamaño a entrenar
val_size<-dim(database)[1]*val_perc #tamaño de subconjuntos
train_size<-ceiling(train_size) #Truncar fracciones hacía arriba
val_size<-floor(val_size) #Truncar fracciones hacía abajo
train<-database[1:train_size,]
val<-database[(train_size+1):(train_size+val_size),]

In [4]:
print(dim(val))
print(dim(train))

[1] 142  11
[1] 572  11


Ahora generaremos una base utilizando el método de k-Fold Cross Validation para $k=4$ y procedemos a también calcular el tamaño de la base $N$.

In [6]:
k<-4
d1<-train[variables]
#t1<-test[variables]

1. Hacemos el modelo logístico para la base y establecemos los parámetros para establecer validación cruzada por método de k-Fold

In [7]:
lrfit = train(Survived ~ ., data=d1, method='glm',
    trControl = trainControl(method = "cv",
                             number=k,p=(1/k)))

Se busca el Accuracy de validación final de la REGRESIÓN LOGÍSTICA

In [8]:
lrPredict <- predict(lrfit,newdata = val)
confusionMatrix(lrPredict, val$Survived )

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 75 10
         1 14 43
                                         
               Accuracy : 0.831          
                 95% CI : (0.759, 0.8886)
    No Information Rate : 0.6268         
    P-Value [Acc > NIR] : 8.459e-08      
                                         
                  Kappa : 0.6442         
 Mcnemar's Test P-Value : 0.5403         
                                         
            Sensitivity : 0.8427         
            Specificity : 0.8113         
         Pos Pred Value : 0.8824         
         Neg Pred Value : 0.7544         
             Prevalence : 0.6268         
         Detection Rate : 0.5282         
   Detection Prevalence : 0.5986         
      Balanced Accuracy : 0.8270         
                                         
       'Positive' Class : 0              
                                         

In [9]:
Acc_lr<-as.numeric(confusionMatrix(lrPredict,
                    val$Survived )$overall[1])
Acc_lr

[1] 0.8309859

2. Ahora para el modelo knn e igualmente establecemos el modelo de cv

In [10]:
Knnfit = train(Survived ~ ., data=d1, method='knn', 
     trControl = trainControl(method = "cv",
                              number=k,p=(1/k)))

Se busca el Accuracy de validación final para K-NEAREST NEIGHBORS

In [11]:
knnPredict <- predict(Knnfit,newdata = val)
confusionMatrix(knnPredict, val$Survived)

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 72 26
         1 17 27
                                          
               Accuracy : 0.6972          
                 95% CI : (0.6145, 0.7714)
    No Information Rate : 0.6268          
    P-Value [Acc > NIR] : 0.04813         
                                          
                  Kappa : 0.3297          
 Mcnemar's Test P-Value : 0.22247         
                                          
            Sensitivity : 0.8090          
            Specificity : 0.5094          
         Pos Pred Value : 0.7347          
         Neg Pred Value : 0.6136          
             Prevalence : 0.6268          
         Detection Rate : 0.5070          
   Detection Prevalence : 0.6901          
      Balanced Accuracy : 0.6592          
                                          
       'Positive' Class : 0               
                                          

In [12]:
Acc_knn<-as.numeric(confusionMatrix(knnPredict,
                        val$Survived )$overall[1])
Acc_knn

[1] 0.6971831

3. Finalmente establecemos el modelo de árbol de decisión

In [13]:
dtfit = train(Survived ~ ., data=d1,method="rpart",
              trControl = trainControl(method = "cv",
                                       number=k,p=(1/k)))

Se busca el Accuracy de validación final para ÁRBOLES DE DECISIÓN

In [14]:
dtPredict <- predict(dtfit,newdata = val)
confusionMatrix(dtPredict, val$Survived )

Confusion Matrix and Statistics

          Reference
Prediction  0  1
         0 76 12
         1 13 41
                                          
               Accuracy : 0.8239          
                 95% CI : (0.7512, 0.8827)
    No Information Rate : 0.6268          
    P-Value [Acc > NIR] : 2.437e-07       
                                          
                  Kappa : 0.6251          
 Mcnemar's Test P-Value : 1               
                                          
            Sensitivity : 0.8539          
            Specificity : 0.7736          
         Pos Pred Value : 0.8636          
         Neg Pred Value : 0.7593          
             Prevalence : 0.6268          
         Detection Rate : 0.5352          
   Detection Prevalence : 0.6197          
      Balanced Accuracy : 0.8138          
                                          
       'Positive' Class : 0               
                                          

In [15]:
Acc_dt<-as.numeric(confusionMatrix(dtPredict, val$Survived )$overall[1])
Acc_dt

[1] 0.8239437

In [16]:
final_decision<-c(Acc_lr,Acc_dt,Acc_knn)
which.max(final_decision)

[1] 1

In [19]:
print(Acc_lr)
print(Acc_dt)
print(Acc_knn)

[1] 0.8309859
[1] 0.8239437
[1] 0.6971831
